In [10]:
import duckdb

In [11]:
con = duckdb.connect()

def cria_tab(con):
    con.sql("DROP TABLE IF EXISTS Usina; DROP SEQUENCE IF EXISTS id_usina; DROP TABLE IF EXISTS Precipitacao;")
    sql_create_usina = """
        CREATE SEQUENCE id_usina;
        CREATE TABLE IF NOT EXISTS Usina (
            id INTEGER PRIMARY KEY,
            nome VARCHAR NOT NULL,
            proprietario VARCHAR,
            munic1 VARCHAR NOT NULL,
            uf1 VARCHAR NOT NULL,
            munic2 VARCHAR,
            uf2 VARCHAR,
            rio VARCHAR NOT NULL,
            potkw FLOAT,
            atualizacao DATE,
            iniciop DATE,
            potfis FLOAT,
            lat DOUBLE PRECISION,
            long DOUBLE PRECISION,
            estagio VARCHAR
        );
    """
    sql_create_precipitacao = """
        CREATE TABLE IF NOT EXISTS Precipitacao AS
        SELECT
            ROW_NUMBER() OVER () AS id,
            RIO AS rio,
            X AS long,
            Y AS lat,
            ANOINI AS iniciop,
            MED_ANUAL AS medanual,
            DP_ANUAL AS dpanual,
            CV_ANUAL AS cvanual,
            MIN_ANUAL AS minanual,
            MAX_ANUAL AS maxanual,
            N_CDD AS ndiasecoscons,
            MAX_CDD AS mxdiasecos,
            MIN_CDD AS mndiasecos,
            N_R20 AS nr20,
            MED_R20 AS mdr20,
            MAX_R20 AS mxr20,
            PMXGEVMMLT AS prec_max_gev
        FROM read_csv_auto('Dados_Pluviométricos_ANA.csv')
    """
    con.sql(sql_create_usina)
    #con.sql(sql_create_precipitacao)
    con.commit()

def carrega_csv(con):
    sql_carrega_tabela_usi = f"""
        INSERT INTO Usina (id, nome, proprietario, munic1, uf1, munic2, uf2, rio, potkw, atualizacao, iniciop, potfis, lat, long, estagio)
        SELECT nextval('id_usina') as id, nome, prop as proprietario, munic1, uf1, munic2, uf2, rio,
        CAST(REPLACE(CAST(potkw AS VARCHAR), ',', '.') AS DOUBLE),
        TRY_CAST(atualiz AS DATE) as atualizacao,
        CASE WHEN iniciop = '0' THEN NULL ELSE STRPTIME(iniciop, '%d/%m/%Y') END as iniciop,
        CAST(REPLACE(CAST(potfisc AS VARCHAR), ',', '.') AS DOUBLE),
        CAST(REPLACE(lat, ',', '.') AS DOUBLE),
        CAST(REPLACE(long, ',', '.') AS DOUBLE),
        estag as estagio
        FROM read_csv(['Centrais_Geradoras_Hidrelétricas_CGH.csv', 'Usinas_Hidrelétricas_UHE.csv'], union_by_name=true,sep=';', dateformat = '%d/%m/%Y')
    """
    sql_clean_usiprec = """DROP TABLE IF EXISTS Usi_Prec_Desnorm; DROP SEQUENCE IF EXISTS id_usi_prec;"""
    sql_create_desnorm_usiprec = """
        CREATE SEQUENCE id_usi_prec;
        CREATE TABLE IF NOT EXISTS Usi_Prec_Desnorm AS
        SELECT u.*, p.medanual, p.dpanual, p.cvanual, p.minanual, p.maxanual, p.ndiasecoscons, p.mxdiasecos, p.mndiasecos, p.nr20, p.mdr20, p.mxr20, p.prec_max_gev
        FROM Usina u
        JOIN Precipitacao p
        ON ABS(u.lat - p.lat) <= 0.08 AND ABS(u.long - p.long) <= 0.08
    """
    con.sql(sql_carrega_tabela_usi)
    #con.sql(sql_clean_usiprec)
    #con.sql(sql_create_desnorm_usiprec)
    con.commit()

def executa_query(con, query):
    print(con.sql(query).show())

In [12]:
cria_tab(con)
carrega_csv(con)

In [13]:
con.sql("SELECT * FROM Usina").show()

┌───────┬───────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────┬─────────┬────────────────┬─────────┬───────────────────┬───────────┬─────────────┬────────────┬───────────┬──────────────┬──────────────┬─────────────────┐
│  id   │                     nome                      │                                                                          proprietario                                                                           │        munic1         │   uf1   │     munic2     │   uf2   │        rio        │   potkw   │ atualizacao │  iniciop   │  potfis   │     lat      │     long     │     estagio     │
│ int32 │                    varchar                    │                                                                             varchar                                                           

In [14]:
# Porcentagem de usinas que abastecem dois estados e dois municípios e as maiores potências (no papel) dessas usinas
query_1A = """  WITH porcentagens AS ( 
                    SELECT (COUNT(CASE WHEN uf1 <> uf2 THEN 1 END) * 100.0 / COUNT(*)) AS porcentagem_multiestado, (COUNT(CASE WHEN munic1 <> munic2 THEN 1 END) * 100.0 / COUNT(*)) AS porcentagem_multimunicipio FROM Usina),
                maior_pot_multiestado AS ( SELECT MAX(potkw) AS maior_pot_teorica_multiestado
                    FROM Usina
                    WHERE uf1 <> uf2
                ),
                maior_pot_multimunicipio AS ( SELECT MAX(potkw) AS maior_pot_teorica_multimunicipio
                    FROM Usina
                    WHERE munic1 <> munic2
                )
                SELECT * FROM porcentagens
                CROSS JOIN maior_pot_multiestado
                CROSS JOIN maior_pot_multimunicipio;"""
executa_query(con, query_1A)

# Novas hidrelétricas (menos de 10 anos) e suas potências
query_2A = """ SELECT nome, potkw, potfis, date_part('year', iniciop) AS ano_inicio, datediff('year', iniciop, DATE '2023-01-01') AS anos_operacao 
    FROM Usina
    WHERE datediff('year', iniciop, DATE '2023-01-01') < 10 AND estagio = 'Operação'
    ORDER BY anos_operacao DESC;"""

# Média de potência por década
query_3A = """ SELECT CAST(FLOOR(extract('year' FROM iniciop) / 10) * 10 AS INTEGER) || 's' AS decada, ROUND(AVG(potkw), 2) AS pot_med
            FROM Usina
            WHERE estagio = 'Operação'
            GROUP BY decada
            ORDER BY decada;"""

# Os rios que geram mais potencial e sua quantidade de usinas e o(s) estado(s) onde se encontram
query_4A = """ SELECT DISTINCT rio, ROUND(SUM(potkw), 2) AS pot_total, COUNT(*) AS quantidade_usinas, uf1
            FROM Usina
            WHERE estagio = 'Operação'
            GROUP BY rio, uf1
            ORDER BY pot_total DESC;"""

# Quantidade de usinas fora de operação por estado
query_5A = """ SELECT uf1, COUNT(*) AS quantidade_usinas
            FROM Usina
            WHERE estagio != 'Operação'
            GROUP BY uf1;"""

┌─────────────────────────┬────────────────────────────┬───────────────────────────────┬──────────────────────────────────┐
│ porcentagem_multiestado │ porcentagem_multimunicipio │ maior_pot_teorica_multiestado │ maior_pot_teorica_multimunicipio │
│         double          │           double           │             float             │              float               │
├─────────────────────────┼────────────────────────────┼───────────────────────────────┼──────────────────────────────────┤
│                 20.3125 │                   53.90625 │                     3444000.0 │                        3444000.0 │
└─────────────────────────┴────────────────────────────┴───────────────────────────────┴──────────────────────────────────┘

None


In [15]:
executa_query(con, query_2A)

┌───────────────────────┬────────┬────────┬────────────┬───────────────┐
│         nome          │ potkw  │ potfis │ ano_inicio │ anos_operacao │
│        varchar        │ float  │ float  │   int64    │     int64     │
├───────────────────────┼────────┼────────┼────────────┼───────────────┤
│ Hidrogerador Quântico │ 1225.0 │ 1225.0 │       2016 │             7 │
│ Jph Participações     │  800.0 │  800.0 │       2017 │             6 │
└───────────────────────┴────────┴────────┴────────────┴───────────────┘

None


In [16]:
executa_query(con, query_3A)

┌─────────┬───────────┐
│ decada  │  pot_med  │
│ varchar │  double   │
├─────────┼───────────┤
│ 1900s   │    857.81 │
│ 1910s   │   7865.34 │
│ 1920s   │  87125.09 │
│ 1930s   │   1219.57 │
│ 1940s   │     332.5 │
│ 1950s   │  14908.52 │
│ 1960s   │ 239522.14 │
│ 1970s   │  638305.0 │
│ 1980s   │ 134543.67 │
│ 1990s   │ 411676.25 │
│ 2000s   │   11558.0 │
│ 2010s   │    1207.4 │
│ NULL    │       0.0 │
├─────────┴───────────┤
│ 13 rows   2 columns │
└─────────────────────┘

None


In [17]:
executa_query(con, query_4A)

┌────────────────────────┬───────────┬───────────────────┬─────────┐
│          rio           │ pot_total │ quantidade_usinas │   uf1   │
│        varchar         │  double   │       int64       │ varchar │
├────────────────────────┼───────────┼───────────────────┼─────────┤
│ Rio Paraná             │ 4995200.0 │                 2 │ SP      │
│ Rio Grande             │ 3190200.0 │                 4 │ SP      │
│ Rio Grande             │ 2030000.0 │                 3 │ MG      │
│ Rio Tietê              │ 1882050.0 │                 9 │ SP      │
│ Rio Paraná             │ 1540000.0 │                 1 │ MS      │
│ Rio Paranapanema       │ 1223956.0 │                 6 │ SP      │
│ Rio Paranapanema       │ 1186800.0 │                 5 │ PR      │
│ Rio Pedras             │  889000.0 │                 1 │ SP      │
│ Rio Juquiá-Guaçu       │  230700.0 │                 6 │ SP      │
│ Rio Pardo              │  222816.0 │                 4 │ SP      │
│     ·                  │       ·

In [18]:
executa_query(con, query_5A)

┌─────────┬───────────────────┐
│   uf1   │ quantidade_usinas │
│ varchar │       int64       │
├─────────┼───────────────────┤
│ SP      │                26 │
└─────────┴───────────────────┘

None
